In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from matplotlib.ticker import PercentFormatter

In [2]:
df = pd.read_csv('flights_NY.csv')

In [3]:
df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance
0,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400
1,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416
2,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089
3,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576
4,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762


In [4]:
df.shape[0]

336776

In [5]:
df = df.dropna()
df.shape[0]

327346

In [ ]:
"""
1. По каждой авиакомпании оцените вероятность того, что задержка прилета будет положительной и
нарисуйте график, показывающий распределение этой вероятности по авиакомпаниям.
"""
carriers = sorted(df['carrier'].unique())
delaysCount = {"carriers": [], "probability": []}
for carrier in carriers:
    carrierFlights = df.loc[df['carrier'] == carrier]
    carriersFlightsDelayed = carrierFlights.loc[carrierFlights['arr_delay'] > 0]
    delaysCount["carriers"].append(carrier)
    delaysCount["probability"].append(carriersFlightsDelayed.shape[0] / carrierFlights.shape[0])

In [ ]:
delayProbability = pd.DataFrame(delaysCount)
ax = delayProbability.plot.bar(x='carriers', y='probability', rot=0)

In [ ]:
"""
2. Постройте гистограмму расстояния перелета distance.
Выделите на ней три группы перелетов: короткие, средние и длинные.
Какие значения distance разумно выбрать в качестве границы, разделяющей короткие.
средние и длинные перелеты? Куда летят самолеты в группе длинных перелетов?
Найдите среднее время задержки вылета в каждой из трех выделенных групп.
"""
df.hist(column='distance', bins=70, density=True)

In [ ]:
# short distance flights < 1200 miles
# middle distance flights from 1200 miles to 3000 miles
# long distance flights > 3000 miles

In [ ]:
longDistanceFlights = df.loc[df['distance'] > 3000]
print(f"Long distance destinations: {longDistanceFlights['dest'].unique()}")

In [ ]:
print("Short distance flights delay:", df.loc[df['distance'] < 1200]['arr_delay'].mean())
print("Medium distance flights delay:", df.loc[(df['distance'] > 1200) & (df['distance'] < 3000)]['arr_delay'].mean())
print("Long distance flights delay:", longDistanceFlights['arr_delay'].mean())

In [ ]:
"""
3. Нарисуйте график среднего времени задержки вылета по месяцам и отметьте на
нем границы доверительных интервалов с уровнем доверия 0.95.
С помощью подходящего статистического теста проверьте, можно ли принять гипотезу
 о равенстве средних в январе и феврале на уровне значимости 0.05?
На уровне значимости 0.01?
"""
delaysByMonth = {"month": list(range(1, 13)), "delay": []}
for month in range(1, 13):
    delaysByMonth["delay"].append(df.loc[df['month'] == month]['arr_delay'].mean())

In [ ]:
delay = pd.DataFrame(delaysByMonth)
ax = delay.plot.bar(x='month', y='delay', rot=0)

In [ ]:
grouped_delays = df.groupby('month')['dep_delay'].mean().reset_index()
grouped_delays.columns = ['month', 'average_delay_time']

std_error = stats.sem(grouped_delays.average_delay_time)

lower_bound = grouped_delays.average_delay_time - std_error  *  stats.t.ppf((1 + 0.975) / 2, len(grouped_delays.average_delay_time) - 1)
upper_bound = grouped_delays.average_delay_time + std_error  *  stats.t.ppf((1 + 0.975) / 2, len(grouped_delays.average_delay_time) - 1)

plt.plot(grouped_delays.average_delay_time, label='Mean values')

# Adding confidence interval boundaries
plt.fill_between(range(len(grouped_delays.average_delay_time)), lower_bound, upper_bound, alpha=0.05, color='blue')
plt.xlabel('Month')
plt.ylabel('Values')
plt.title('Confidence Intervals with Mean Values')
plt.show()

In [ ]:
grouped_delays.head()

In [ ]:
january_data = grouped_delays.loc[grouped_delays['month'] == 1]['average_delay_time']
february_data = grouped_delays.loc[grouped_delays['month'] == 2]['average_delay_time']
# Проверяем гипотезу на уровне значимости 0.05
t_stat, p_value = stats.ttest_ind(january_data, february_data)
print("При уровне значимости 0.01 гипотеза о равенстве средних " + ("принимается." if p_value > 0.05 else "отвергается."))
# Проверяем гипотезу на уровне значимости 0.01
t_stat, p_value = stats.ttest_ind(january_data, february_data, alternative='less')
print("При уровне значимости 0.01 гипотеза о равенстве средних " + ("принимается." if p_value > 0.01 else "отвергается."))


In [ ]:
""" 4. Найдите коэффициент корреляции между расстоянием distance и временем полета air_time.
Постройте точечную диаграмму в осях distance (х) и air_time (y).
Найдите коэффициенты линейной регрессии и нанесите полученную прямую на график.
Как можно интерпретировать смысл коэффициентов полученной линейной модели?
"""
ax1 = df.plot.scatter(x='distance', y='air_time')

In [ ]:
# calculate correlation coefficient = (mean(xy) - mean(x) * mean(y)) / (deviation(x) * deviation(y))
x_y_mean = 0
x_mean = 0
y_mean = 0
x_deviation = 0
y_deviation = 0
N = df.shape[0]
for x, y in zip(df['distance'], df['air_time']):
    x_y_mean += x * y
    x_mean += x
    y_mean += y
    x_deviation += x ** 2
    y_deviation += y ** 2
    
x_y_mean /= N
x_mean /= N
y_mean /= N
x_deviation /= N
x_deviation = (x_deviation - x_mean ** 2) ** 0.5
y_deviation /= N
y_deviation = (y_deviation - y_mean ** 2) ** 0.5
print(f"Correlation coefficient: {(x_y_mean - x_mean * y_mean) / (x_deviation * y_deviation)}")

In [ ]:
# check correlation coefficient using pandas method .corr()
df[["distance", "air_time"]].corr(method='pearson')
# results are the same

In [ ]:
# to find coefficitents of a linear regression, we will use the method of least squares
x_sum = 0
squared_x_sum = 0
y_sum = 0
x_y_sum = 0
N = df.shape[0]
for x, y in zip(df['distance'], df['air_time']):
    x_sum += x
    squared_x_sum += x ** 2
    y_sum += y
    x_y_sum += x * y
# according to the Cramer's rule
det = (squared_x_sum * N) - (x_sum ** 2)
det_a = (x_y_sum * N) - (x_sum * y_sum)
a = det_a / det

det_b = (squared_x_sum * y_sum) - (x_y_sum * x_sum)
b = det_b / det
print(f"Coefficients of a linear regression y = ax + b: {a}, {b}")

In [ ]:
for key, el in df.head(10).iterrows():
    pred = el["distance"] * a + b
    print(f'x: {el["distance"]}, y: {el["air_time"]} ->  prediction: {pred}, relative error(%): {abs(pred - el["air_time"]) / el["air_time"] * 100}')

In [ ]:
x = list(range(df["distance"].min(), df["distance"].max()))
plt.plot(x, [a*el + b for el in x], color="orange", linewidth=4)
plt.scatter(df["distance"], df["air_time"])

plt.xlabel('Distance (miles)')
plt.ylabel('Air time (min)')
plt.show()

In [ ]:
"""
Интерпретация смысла коэффициентов линейной регрессии: коэффициент наклона ≈ 0.12,
таким образом при увеличении дистанции рейса, например, на 1000 миль время рейса увеличится на 120 минут.
То есть, средняя скорость самолета ≈ 500 миль/ч.
В случае если, дистанция рейса = 0 миль, значение линейной регрессии 0.12 * 0 + 18 = 18 минут.
Можно предположить, о том что среднее время подготовки рейса (инструктаж по правилам безопасности, выезд на ВПП и т.д) равняется 18 минут.
"""

In [ ]:
""" 5. Постройте нормированную гистограмму распределения задержки прилета по
тем рейсам, которые вылетели в пределах +/-15 минут от времени в расписании.
Сделайте предположение о том, каким распределением может описываться полученная
гистограмма, оцените параметры этого распределения и нанесите график плотности
на график с гистограммой.
"""

df = df.loc[(df['dep_delay'] >= -15) & (df['dep_delay'] <= 15)]
df = df.reset_index(drop=True)

In [ ]:
sns.histplot(df['arr_delay'], kde=True, bins=int(180/2))
# Percent formatting 
plt.gca().yaxis.set_major_formatter(PercentFormatter(20000))
# Оси
plt.title('Arrival delay diagram')
plt.xlabel('Delay (min)')
plt.ylabel('% of flights')

In [ ]:
# Calculating distribution parameters
m = df['arr_delay'].mean().round(2)
v = df['arr_delay'].var().round(2)

In [ ]:
print("Нормальное распределение с матожиданием", m, ", дисперсией", v, "(мин)")

In [ ]:
""" 6. Постройте модель логистической регрессии, которая будет предсказывать, что задержка прилета составит более 15 минут.
Какие признаки вы будете использовать?
Какие новые признаки, на основе имеющихся, можете предложить?
Какова точность полученной модели (precision, recall, accuracy)?
"""

In [ ]:
""" Исключаем следующие признаки:
1) dep_time, arr_time, так как время вылета в общем случае не будет влиять на задержку
2) arr_delay, значения данного столбца были преобразованы в arr_delay_binary (0, если задержка меньше 15 минут, 1 в противном случае) и использованы как ответы на задачу
3) tailnum
4) day, year, так как данные значения не должны играть существенной роли, в отличие от месяца (в разные месяцы в среднем разная погода, что может сказаться на задержках)
5) flight, origin, dest. Опять же, данные не играют роли.
Также можно было добавить признак time_of_day, так как, возможно, в разные временные промежутки дня статистика будет меняться.
"""

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score

data = pd.read_csv('flights_NY.csv')
data = data.dropna()

# add new binary feature: 1 if df["arr_delay"] > 15 else 0
data['arr_delay_binary'] = (data['arr_delay'] > 15).astype(int)

data = data.drop('dep_time', axis=1)
data = data.drop('arr_time', axis=1)
data = data.drop('arr_delay', axis=1)
data = data.drop('tailnum', axis=1)
data = data.drop('day', axis=1)
data = data.drop('year', axis=1)
data = data.drop('flight', axis=1)
data = data.drop('origin', axis=1)
data = data.drop('dest', axis=1)

# factorize carrier
data['carrier'], uniques = pd.factorize(data['carrier'])

X = data.drop('arr_delay_binary', axis=1)
y = data['arr_delay_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=85)

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# evaluating model
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)